In [1]:
import requests
#import requests_cache
from bs4 import BeautifulSoup
import pandas as pd
import json
import numpy as np
from tqdm import tqdm_notebook  as tqdm
import time

In [2]:
##分析

In [3]:
#csv読み込み
df = pd.read_csv('player_data.csv')
df.head()

,ID,選手名,年,チーム,試合,打席,打数,得点,安打,二塁打,...,盗塁刺,犠打,犠飛,四球,死球,三振,併殺打,打率,長打率,出塁率
0,21825130,山﨑 福也,2015,オリックス,17,3,3,0,1,0,...,0,0,0,0,0,0,0,0.333,0.333,0.333
1,21825130,山﨑 福也,2016,オリックス,17,2,2,0,0,0,...,0,0,0,0,0,0,0,0.000,0.000,0.000
2,21825130,山﨑 福也,2017,オリックス,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000,0.000,0.000
3,21825130,山﨑 福也,2018,オリックス,7,3,3,0,1,0,...,0,0,0,0,0,2,0,0.333,0.333,0.333
4,21825130,山﨑 福也,2019,オリックス,36,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000,0.000,0.000


In [4]:
#wba計算
df_ind=df
df_ind = df_ind.loc[df_ind.年 >2018]
df_ind["単打"]=df_ind["安打"]-df_ind["二塁打"]-df_ind["三塁打"]-df_ind["本塁打"]
df_ind["wOBA分子"]=0.7*(df_ind["四球"]+df_ind["死球"])+0.9*df_ind["単打"]+1.3*df_ind["二塁打"]+1.6*df_ind["三塁打"]+2*df_ind["本塁打"]
df_ind["wOBA分母"]=df_ind["打数"]+df_ind["犠飛"]+df_ind["死球"]+df_ind["四球"]
df_ind["wOBA"]=df_ind["wOBA分子"]/df_ind["wOBA分母"]
df_ind=df_ind.fillna(0)
df_ind.head()

<ipython-input-4-412dc8f5a89a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind["単打"]=df_ind["安打"]-df_ind["二塁打"]-df_ind["三塁打"]-df_ind["本塁打"]
<ipython-input-4-412dc8f5a89a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind["wOBA分子"]=0.7*(df_ind["四球"]+df_ind["死球"])+0.9*df_ind["単打"]+1.3*df_ind["二塁打"]+1.6*df_ind["三塁打"]+2*df_ind["本塁打"]
<ipython-input-4-412dc8f5a89a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,ID,選手名,年,チーム,試合,打席,打数,得点,安打,二塁打,...,死球,三振,併殺打,打率,長打率,出塁率,単打,wOBA分子,wOBA分母,wOBA
4,21825130,山﨑 福也,2019,オリックス,36,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0
5,21825130,山﨑 福也,2020,オリックス,15,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0
6,71875151,宮城 大弥,2020,オリックス,3,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0
12,71675139,吉田 一将,2019,オリックス,33,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0
13,71675139,吉田 一将,2020,オリックス,23,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0


In [5]:
#チームOBA（年）
df_team = pd.read_csv('team_result.csv')
df_team["単打"]=df_team["安打"]-df_team["二塁打"]-df_team["三塁打"]-df_team["本塁打"]
df_team["wOBA分子"]=0.7*(df_team["四球"]+df_team["死球"])+0.9*df_team["単打"]+1.3*df_team["二塁打"]+1.6*df_team["三塁打"]+2*df_team["本塁打"]
df_team["wOBA分母"]=df_team["打数"]+df_team["犠飛"]+df_team["死球"]+df_team["四球"]
df_team["wOBA"]=df_team["wOBA分子"]/df_team["wOBA分母"]
#df=df.fillna(0)
df_team

,年,チーム,試合,打席,打数,得点,安打,二塁打,三塁打,本塁打,...,死球,三振,併殺打,打率,長打率,出塁率,単打,wOBA分子,wOBA分母,wOBA
0,2019,埼玉西武,143,5619,4898,756,1299,229,23,174,...,49,1083,88,0.265,0.428,0.344,873,1891.7,5541,0.341400
1,2019,千葉ロッテ,143,5503,4789,642,1194,214,21,158,...,87,1013,99,0.249,0.402,0.330,801,1763.1,5410,0.325896
2,2019,東北楽天,143,5506,4766,614,1196,195,22,141,...,68,1127,114,0.251,0.390,0.333,838,1749.8,5408,0.323558
3,2019,福岡ソフトバンク,143,5345,4775,582,1199,172,19,183,...,43,1084,85,0.251,0.410,0.312,825,1670.5,5245,0.318494
4,2019,北海道日本ハム,143,5428,4778,560,1200,211,28,93,...,43,1044,106,0.251,0.365,0.325,868,1663.6,5350,0.310953
5,2019,オリックス,143,5391,4774,544,1153,181,22,102,...,66,1030,85,0.242,0.353,0.309,848,1575.8,5290,0.297883
6,2019,読 売,143,5537,4835,663,1242,221,15,183,...,47,1205,106,0.257,0.422,0.335,823,1828.9,5453,0.335393
7,2019,東京ヤクルト,143,5572,4833,656,1181,205,19,167,...,56,1212,105,0.244,0.398,0.329,790,1780.1,5488,0.324362
8,2019,横浜DeNA,143,5375,4778,596,1177,210,14,163,...,41,1065,109,0.246,0.398,0.315,790,1675.4,5294,0.316471
9,2019,広島東洋,143,5504,4852,591,1232,216,17,140,...,36,1106,86,0.254,0.392,0.324,859,1721.6,5393,0.319229


In [6]:
#リーグwOBA
df_league = pd.read_csv('league_result.csv')
df_league["単打"]=df_league["安打"]-df_league["二塁打"]-df_league["三塁打"]-df_league["本塁打"]
df_league["wOBA分子"]=0.7*(df_league["四球"]+df_league["死球"])+0.9*df_league["単打"]+1.3*df_league["二塁打"]+1.6*df_league["三塁打"]+2*df_league["本塁打"]
df_league["wOBA分母"]=df_league["打数"]+df_league["犠飛"]+df_league["死球"]+df_league["四球"]
df_league["wOBA"]=df_league["wOBA分子"]/df_league["wOBA分母"]
df_league=df_league.rename(columns={'試合': 'リーグ試合'}) 
df_league2=df_league.drop(df_league.loc[:,'リーグ試合':'wOBA分母'],axis=1)
#df=df.fillna(0)
df_league2=df_league2.rename(columns={'wOBA': 'リーグwOBA'}) 
df_league=df_league.rename(columns={'wOBA': 'リーグwOBA','得点':'リーグ得点','打席':'リーグ打席'}) 
df_league3=df_league.drop("打数",axis=1)

df_league3

,年,リーグ,リーグ試合,リーグ打席,リーグ得点,安打,二塁打,三塁打,本塁打,塁打,...,死球,三振,併殺打,打率,長打率,出塁率,単打,wOBA分子,wOBA分母,リーグwOBA
0,2019,パ,858,32792,3698,7241,1202,135,851,11266,...,356,6381,577,1.509,2.348,1.953,5053,10314.5,32244,0.319889
1,2019,セ,858,32803,3607,7310,1308,116,837,11361,...,277,6656,612,1.515,2.353,1.939,5049,10281.8,32231,0.319003
2,2020,パ,720,27080,2963,5797,1006,122,614,8889,...,265,5368,463,1.476,2.262,1.952,4055,8381.8,26581,0.315330
3,2020,セ,720,27093,2962,6095,1069,95,674,9376,...,218,5429,544,1.523,2.343,1.942,4257,8501.8,26687,0.318575


In [7]:
#リーグ分類
team_league = pd.read_csv('team_league.csv')
team_league

,チーム,リーグ
0,埼玉西武,パ
1,千葉ロッテ,パ
2,東北楽天,パ
3,福岡ソフトバンク,パ
4,北海道日本ハム,パ
5,オリックス,パ
6,読 売,セ
7,東京ヤクルト,セ
8,横浜DeNA,セ
9,広島東洋,セ


In [8]:
#リーグ結合
df_ind_j = pd.merge(df_ind,team_league, how="left", on=['チーム'])
#リーグOBA
df_ind_j = pd.merge(df_ind_j,df_league2, how="left", on=['リーグ','年'])
#打撃得点
df_ind_j["打撃得点"]=(df_ind_j["wOBA"]-df_ind_j["リーグwOBA"])*(df_ind_j["打席"]+df_ind_j["犠飛"]+df_ind_j["四球"]+df_ind_j["死球"])/1.2
#リーグ総得点・リーグ総打席
df_ind_j=pd.merge(df_ind_j, df_league3.loc[:, "年":"リーグ得点"], how='left', on=['リーグ','年']) 
#wrc
df_ind_j["wRC"]=df_ind_j["打撃得点"]+df_ind_j["リーグ得点"]/df_ind_j["リーグ打席"]*df_ind_j["打席"]
df_ind_j=df_ind_j.sort_values('wRC', ascending=False)

df_ind_j.head()

,ID,選手名,年,チーム,試合,打席,打数,得点,安打,二塁打,...,wOBA分子,wOBA分母,wOBA,リーグ,リーグwOBA,打撃得点,リーグ試合,リーグ打席,リーグ得点,wRC
210,81785137,鈴木 誠也,2019,広島東洋,140,612,499,112,167,31,...,270.5,612,0.441993,セ,0.319003,74.306471,858,32803,3607,141.601657
297,91895133,山田 哲人,2019,東京ヤクルト,142,641,520,102,141,35,...,264.1,641,0.412012,セ,0.319003,59.060738,858,32803,3607,129.544749
413,51955114,坂本 勇人,2019,読 売,143,639,555,103,173,26,...,265.4,636,0.417296,セ,0.319003,58.975292,858,32803,3607,129.239384
960,31835133,柳田 悠岐,2020,福岡ソフトバンク,119,515,427,90,146,23,...,235.5,515,0.457282,パ,0.315330,71.330405,720,27080,2963,127.679925
109,63165132,吉田 正尚,2019,オリックス,143,610,521,92,168,24,...,252.9,610,0.414590,パ,0.319889,55.163445,858,32792,3698,123.954003


In [9]:
df_ind_j.to_csv('ana.csv', encoding='cp932')

In [10]:
#投手csv読み込み
df_p = pd.read_csv('member_p.csv',names=('team', 'ID', '選手名'))
#df_p["ID"]
for p_id in df_p["ID"]:
        df_ind_j=df_ind_j.drop(df_ind_j.index[df_ind_j["ID"] == p_id])
        
df_ind_j.head()

,ID,選手名,年,チーム,試合,打席,打数,得点,安打,二塁打,...,wOBA分子,wOBA分母,wOBA,リーグ,リーグwOBA,打撃得点,リーグ試合,リーグ打席,リーグ得点,wRC
210,81785137,鈴木 誠也,2019,広島東洋,140,612,499,112,167,31,...,270.5,612,0.441993,セ,0.319003,74.306471,858,32803,3607,141.601657
297,91895133,山田 哲人,2019,東京ヤクルト,142,641,520,102,141,35,...,264.1,641,0.412012,セ,0.319003,59.060738,858,32803,3607,129.544749
413,51955114,坂本 勇人,2019,読 売,143,639,555,103,173,26,...,265.4,636,0.417296,セ,0.319003,58.975292,858,32803,3607,129.239384
960,31835133,柳田 悠岐,2020,福岡ソフトバンク,119,515,427,90,146,23,...,235.5,515,0.457282,パ,0.315330,71.330405,720,27080,2963,127.679925
109,63165132,吉田 正尚,2019,オリックス,143,610,521,92,168,24,...,252.9,610,0.414590,パ,0.319889,55.163445,858,32792,3698,123.954003


In [11]:
#リプレース補正
df_rep=df_ind_j.loc[df_ind_j.年 ==2019]
df_rep=df_rep.loc[df_rep.打席 <200]
df_rep2=df_rep.groupby("チーム").sum()
df_rep2["R-wOBA分子"]=0.7*(df_rep2["四球"]+df_rep2["死球"])+0.9*df_rep2["単打"]+1.3*df_rep2["二塁打"]+1.6*df_rep2["三塁打"]+2*df_rep2["本塁打"]
df_rep2["R-wOBA分母"]=df_rep2["打数"]+df_rep2["犠飛"]+df_rep2["死球"]+df_rep2["四球"]
df_rep2["R-wOBA"]=df_rep2["wOBA分子"]/df_rep2["wOBA分母"]
df_rep2.loc[:, "打数":"併殺打"]=df_rep2.loc[:, "打数":"併殺打"].div(df_rep2["打席"], axis=0).mul(50)
df_rep2["打席"]=50
df_rep3=df_rep.sum()
df_rep3.loc["打数":"併殺打"]=df_rep3.loc["打数":"併殺打"].div(df_rep3["打席"], axis=0).mul(50)
df_rep3["打席"]=50
#df=df.fillna(0)
df_rep3

ID                                                9206990882
選手名        柳田　悠岐アルモンテ福田　秀平宗　佑磨岡　大海安達　了一柴田　竜拓川島　慶三長野　久義メヒア...
年                                                     397743
チーム        福岡ソフトバンク中　日福岡ソフトバンクオリックス千葉ロッテオリックス横浜DeNA福岡ソフトバ...
試合                                                      6669
打席                                                        50
打数                                                   44.4992
得点                                                   5.17632
安打                                                   9.69974
二塁打                                                  1.67331
三塁打                                                 0.242396
本塁打                                                  0.90312
塁打                                                   14.5672
打点                                                   4.33185
盗塁                                                  0.989131
盗塁刺                                                 0.347955
犠打                      

In [12]:
#2019
df_ind_2019=df_ind_j.loc[df_ind_j.年 ==2019]
df_ind_2019=df_ind_2019.rename(columns = lambda x: x + '_2019')

df_ind_2020=df_ind_j.loc[df_ind_j.年 ==2020]
df_ind_2020=df_ind_2020.rename(columns = lambda x: x + '_2020')


#df_ind_2020.head()

In [13]:
#2019,2020ジョイン
df_ind_1920 = pd.merge(df_ind_2020,df_ind_2019, how="left", left_on=["ID_2020","選手名_2020"],right_on=["ID_2019","選手名_2019"])

#行削除のちのち削除する
df_ind_1920=df_ind_1920.dropna(how='any')

#リプレースメント補正
#1打席以上50打席以下
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '打数_2019'] = df_ind_1920['打数_2019']+df_rep3["打数"]
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '安打_2019'] = df_ind_1920['安打_2019']+df_rep3["安打"]
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '二塁打_2019'] = df_ind_1920['二塁打_2019']+df_rep3["二塁打"]
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '三塁打_2019'] = df_ind_1920['三塁打_2019']+df_rep3["三塁打"]
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '本塁打_2019'] = df_ind_1920['本塁打_2019']+df_rep3["本塁打"]
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '塁打_2019'] = df_ind_1920['塁打_2019']+df_rep3["塁打"]
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '犠飛_2019'] = df_ind_1920['犠飛_2019']+df_rep3["犠飛"]
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '犠打_2019'] = df_ind_1920['犠打_2019']+df_rep3["犠打"]
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '四球_2019'] = df_ind_1920['四球_2019']+df_rep3["四球"]
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '死球_2019'] = df_ind_1920['死球_2019']+df_rep3["死球"]
df_ind_1920.loc[df_ind_1920['打席_2019'] < 100, '打席_2019'] = df_ind_1920['打席_2019']+df_rep3["打席"]




#2019年の打席
df_ind_1920["打席比率"]=df_ind_1920["打席_2020"]/df_ind_1920["打席_2019"]
df_ind_1920=df_ind_1920.join(df_ind_1920.loc[:, "打席_2019":"併殺打_2019"].mul(df_ind_1920["打席比率"], axis=0).add_prefix('p_'))


#wrc計算
df_ind_1920["p_単打_2019"]=df_ind_1920["p_安打_2019"]-df_ind_1920["p_二塁打_2019"]-df_ind_1920["p_三塁打_2019"]-df_ind_1920["p_本塁打_2019"]
df_ind_1920["p_wOBA分子_2019"]=0.7*(df_ind_1920["p_四球_2019"]+df_ind_1920["p_死球_2019"])+0.9*df_ind_1920["p_単打_2019"]+1.3*df_ind_1920["p_二塁打_2019"]+1.6*df_ind_1920["p_三塁打_2019"]+2*df_ind_1920["p_本塁打_2019"]
df_ind_1920["p_wOBA分母_2019"]=df_ind_1920["p_打数_2019"]+df_ind_1920["p_犠飛_2019"]+df_ind_1920["p_死球_2019"]+df_ind_1920["p_四球_2019"]
df_ind_1920["p_wOBA_2019"]=df_ind_1920["p_wOBA分子_2019"]/df_ind_1920["p_wOBA分母_2019"]
df_ind_1920["p_打撃得点_2019"]=(df_ind_1920["p_wOBA_2019"]-df_ind_1920["リーグwOBA_2019"])*(df_ind_1920["p_打席_2019"]+df_ind_1920["p_犠飛_2019"]+df_ind_1920["p_四球_2019"]+df_ind_1920["p_死球_2019"])/1.2
df_ind_1920["p_wRC_2019"]=df_ind_1920["p_打撃得点_2019"]+df_ind_1920["リーグ得点_2019"]/df_ind_1920["リーグ打席_2019"]*df_ind_1920["p_打席_2019"]



df_ind_1920["伸び"]=df_ind_1920["wRC_2020"]-df_ind_1920["p_wRC_2019"]
df_ind_1920=df_ind_1920.sort_values('伸び', ascending=False)
df_ind_1920=df_ind_1920.dropna(how='any')

df_ind_1920.to_csv('ana_join.csv', encoding='cp932')
df_ind_1920

,ID_2020,選手名_2020,年_2020,チーム_2020,試合_2020,打席_2020,打数_2020,得点_2020,安打_2020,二塁打_2020,...,p_死球_2019,p_三振_2019,p_併殺打_2019,p_単打_2019,p_wOBA分子_2019,p_wOBA分母_2019,p_wOBA_2019,p_打撃得点_2019,p_wRC_2019,伸び
1,41845136,村上 宗隆,2020,東京ヤクルト,120,515,424,70,130,30,...,4.342327,159.797639,7.816189,53.844857,181.596121,515.000000,0.352614,16.419069,73.048189,42.221720
40,61365112,Ｔ－岡田,2020,オリックス,100,377,328,36,84,18,...,1.821546,67.575472,10.669811,42.255701,85.636951,372.536516,0.229875,-31.012732,11.502089,41.349388
26,61965131,堂林 翔太,2020,広島東洋,111,451,401,55,112,21,...,2.624814,46.125000,0.000000,66.014681,111.613756,434.318203,0.256986,-24.962479,24.629235,37.070411
39,61965139,田中 広輔,2020,広島東洋,112,454,378,51,95,11,...,2.557746,113.819718,5.115493,56.270423,110.366761,448.884507,0.245869,-30.786877,19.134716,33.810581
12,21325134,大山 悠輔,2020,阪 神,116,471,423,66,122,21,...,4.011925,78.633731,9.628620,73.017036,148.601704,470.197615,0.316041,-1.257859,50.533044,33.077821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,81085115,中村 剛也,2020,埼玉西武,79,297,258,32,55,14,...,2.132855,65.585278,6.398564,43.723519,113.787792,297.000000,0.383124,17.364623,50.857731,-20.956588
124,13315133,バレンティン,2020,福岡ソフトバンク,60,218,191,16,32,7,...,0.465812,54.500000,5.123932,32.141026,85.476496,218.000000,0.392094,14.923700,38.894861,-24.165915
37,41245130,外崎 修汰,2020,埼玉西武,120,500,433,62,107,18,...,4.025765,106.280193,7.246377,70.048309,179.227053,488.727858,0.366722,21.838878,78.224582,-24.612396
42,91895133,山田 哲人,2020,東京ヤクルト,94,384,334,52,85,17,...,3.594384,72.486739,8.386895,39.538222,158.212793,384.000000,0.412012,35.381160,77.605591,-27.649072


In [14]:
#チームごとの集計
df_team_sum=df_ind_1920.groupby("チーム_2020").sum()
df_team_sum.to_csv('team_r.csv', encoding='cp932')
df_team_sum

,ID_2020,年_2020,試合_2020,打席_2020,打数_2020,得点_2020,安打_2020,二塁打_2020,三塁打_2020,本塁打_2020,...,p_死球_2019,p_三振_2019,p_併殺打_2019,p_単打_2019,p_wOBA分子_2019,p_wOBA分母_2019,p_wOBA_2019,p_打撃得点_2019,p_wRC_2019,伸び
チーム_2020,,,,,,,,,,,,,,,,,,,,,
オリックス,1480173712,56560,1376,4035,3573,415,893,150,22,83,...,50.119624,641.926072,63.779790,620.413367,1160.845107,3953.737914,7.711915,-93.982825,360.293390,63.791044
中 日,1097672879,44440,1336,3921,3502,401,912,164,16,67,...,41.475102,610.751171,72.307078,712.462757,1272.936928,3873.165871,6.514487,33.587188,464.918176,-56.686415
北海道日本ハム,1058738006,46460,1563,4577,3971,486,981,151,15,88,...,41.389228,791.515379,84.850591,715.753742,1414.010066,4520.594703,6.554557,-26.791416,488.286339,-4.943281
千葉ロッテ,977747763,42420,1172,3795,3225,396,773,146,10,82,...,55.095893,658.524023,57.238218,534.950783,1196.490615,3734.990519,6.260347,2.667699,430.525525,-5.622524
埼玉西武,769732371,36360,1090,3819,3339,408,789,131,15,88,...,30.229708,702.498465,59.251765,579.138661,1248.969817,3757.465572,5.381454,46.158765,476.832771,-91.933284
広島東洋,956627732,42420,1287,3798,3351,439,906,178,14,95,...,26.117560,613.712704,51.209031,597.203213,1189.999072,3737.596669,6.256843,2.071912,419.779468,65.781508
東京ヤクルト,1173763351,52520,1413,3899,3395,434,837,145,11,113,...,43.725270,697.628936,68.567575,535.870451,1192.676017,3849.137009,7.537988,-30.449753,398.408700,56.019336
東北楽天,1459453685,56560,1277,4050,3486,495,907,163,17,111,...,52.740161,716.300215,59.944362,622.820525,1333.518359,3976.896561,8.162264,58.508646,514.931969,15.146149
横浜DeNA,1208982886,44440,1322,3904,3532,464,972,158,9,114,...,27.075506,639.414774,82.944580,573.665138,1230.446501,3861.655598,6.547631,-1.123785,428.287277,54.923583
